# Q&A  with the Fed Minutes
### Load document store and embeddings


In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
model_name = 'sentence-transformers/all-mpnet-base-v2'
model_kwargs = {'device':'mps'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)
vector_s = FAISS.load_local("fed_minutes_vector_store", embeddings)

query = 'What happened with the economy in 1968?'
docs_ans = vector_s.similarity_search(query, k=5)
print(docs_ans)

[Document(page_content='8 / 18 / 70 - 44 nature of current policy. he thought the desk and the staff were to be commended for the manner in which they had adapted to the new type of directive, and he personally would be unhappy if the committee were to return to directives of the old type. mr. francis commented that it had been popular to criti cize stabilization actions and the performance of the economy over the past year. however, given the strong inflationary momentum gradually built up from 1964 through 1968, he believed that stabi lization actions had, on the whole, been applied satisfactorily and that the economy had performed as well as could reasonably have been expected. the rate of price increase had not been slowed much, if at all, mr. francis remarked. however, the rise had stopped accelerating, and all econometric models now indicated that a moderation of the upward price movement was likely this fall. cutbacks in real output had been much less than in other periods when 

### Add a generative model to compose the
Load the env with the open AI key

In [13]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('.env.rtf')) # read local .env file

In [7]:
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import langchain
langchain.debug = True

In [14]:
ask_me_about_fed_stuff = RetrievalQA.from_chain_type(llm=OpenAI(model_name='curie'),
                                chain_type="stuff",
                                retriever=vector_s.as_retriever(search_kwargs={"k": 5}),
                                return_source_documents=True)

In [20]:
result = ask_me_about_fed_stuff({'query':'When did Ben Bernanke start participating in the FOMC meetings?'})
print(result)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "When did Ben Bernanke start participating in the FOMC meetings?"
}
[chain/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "question": "When did Ben Bernanke start participating in the FOMC meetings?",
  "context": "voting for this action : messrs. bernanke and dudley, ms. duke, messrs. evans, kohn, lacker, lockhart, tarullo, and warsh, and ms. yellen. voting against this action : none. it was agreed that the next meeting of the committee would be held on tuesday - wednesday, november 3 - 4, 2009. the meeting adjourned at 12 : 35 p. m. on sep - tember 23, 2009. notation votes by notation vote completed on august 28, 2009, the committee unanimously approved the designation of matthew m. luecke as the committee \u2019 s chief freedom of info

In [21]:
result['result']

' Ben Bernanke started participating in the FOMC meetings in August 2005.'

Let's change the prompt

In [92]:
from langchain.prompts import PromptTemplate
prompt_template = """You are a research analyst at a federal reserve bank and you are trying to answer questions or provide colour on statements. Use the following pieces of context to answer the question at the end. Explain the rationale behind your answer.
If you don't have all the elements to answer the query, say it explicitly.

{context}

Question: {question}
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [93]:
from langchain.chat_models import ChatOpenAI
ask_me_about_fed_stuff = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name='gpt-3.5-turbo'),
                                                     chain_type="stuff",
                                                     retriever=vector_s.as_retriever(search_kwargs={"k": 7}),
                                                     return_source_documents=True,
                                                     chain_type_kwargs={'prompt': PROMPT})

In [97]:
result = ask_me_about_fed_stuff({'query':'Views on monetary policy during March 26-27, 1984.'})
print(result['result'])

Based on the summary of economic projections from the FOMC meeting held on September 15-16, 2020, meeting participants expected to continue assessing realized and expected economic conditions relative to the committee's maximum-employment objective and its symmetric 2 percent inflation objective when determining the stance of monetary policy. They revised up their economic outlook due to the stronger-than-expected rebound in economic activity over recent months, but remained attentive to the effects of the COVID-19 pandemic and the measures taken to contain it. Almost all participants projected that real GDP would contract in 2020, but would grow faster than their estimates of its longer-run normal growth rate from 2021 to 2023. They projected that the unemployment rate would decline gradually during the forecast period, with a full economic recovery taking some time. Therefore, it can be inferred that the views on monetary policy during 2021 will likely be determined by the evolution 

In [99]:
result = ask_me_about_fed_stuff({'query':'Summarize the discussion around the selection of a reserve bank to operate the system account on March 26-27, 1984.'})
result['result']

'During the meeting held on March 26-27, 1984, it was agreed that the Federal Reserve Bank of New York would operate the system account. However, there was dissent from Messrs. Martin and Rice and Ms. Seger. It was emphasized that the authority for the selection of the manager and the special manager was vested in the open market committee. The meeting also discussed the uncertainties associated with the introduction of the system of more contemporaneous reserve requirements, and the possibility that depository institutions might desire to hold more excess reserves during the transition period. The next meeting of the committee was scheduled for March 27, 1984.'

In [108]:
result = ask_me_about_fed_stuff({'query':'What was the economic outlook during the FOMC meeting of March 1968? Please include some of the political developments discussed in the meeting and why they were important to the economic outlook.'})
result['result']

"During the FOMC meeting of March 5, 1968, the high degree of uncertainty associated with key variables that would ultimately shape the contours of the economy in 1968 was discussed. These included such factors as the mix of fiscal and monetary policy, the stubbornly high savings rate, the uncertain trend of federal spending for defense and other purposes, the upward movement of prices, and the interaction of those and other variables on spending and investing decisions. Despite such uncertainties, the group forecast remained highly optimistic for the near term. The median forecasts of the production index and gross national product were considerably higher than last fall, with a strong upward thrust in the first half and a more moderate advance in the second half. For 1968 as a whole, industrial production was expected to average 165, a 4 - 1/2 per cent increase over 1967; and GNP was expected to amount to $845 billion, a 7 - 1/2 per cent advance, with an undesirably large portion of 

In [126]:
import re
pattern = re.compile(r'\d{8}')
dates =[pattern.findall(doc.metadata['source'])[0] for doc in result['source_documents']]
print(dates[1])

19680402


In [135]:
def get_date(date: str):
    import pandas as pd
    date = pd.to_datetime(date)
    date = date.strftime("%d%m%Y")
    return date

print(get_date('15 Mar 1968'))




15031968


In [136]:
query = 'hat was the economic outlook during the FOMC meeting of March 1968? Please include some of the political developments discussed in the meeting and why they were important to the economic outlook.?'
docs_ans = vector_s.similarity_search(query, k=7)

In [144]:
docs_dates = [doc.metadata['source'][-12:-4] for doc in docs_ans]
docs_dates



['19680206',
 '19680402',
 '19680419',
 '19680402',
 '19680716',
 '19680819',
 '19671212']

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
model_name = 'sentence-transformers/all-mpnet-base-v2'
model_kwargs = {'device':'mps'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)
vector_s = FAISS.load_local("fed_minutes_vector_store", embeddings)

query = 'What happened with the economy in 1968?'
docs_ans = vector_s.similarity_search(query, k=5)
print(docs_ans)

### Add a generative model to compose the
Load the env with the open AI key

In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('.env.rtf')) # read local .env file

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import langchain
langchain.debug = True

In [ ]:
ask_me_about_fed_stuff = RetrievalQA.from_chain_type(llm=OpenAI(model_name='curie'),
                                chain_type="stuff",
                                retriever=vector_s.as_retriever(search_kwargs={"k": 5}),
                                return_source_documents=True)

In [ ]:
result = ask_me_about_fed_stuff({'query':'When did Ben Bernanke start participating in the FOMC meetings?'})
print(result)

In [ ]:
result['result']

Let's change the prompt

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """You are a research analyst at a federal reserve bank and you are trying to answer questions or provide colour on statements. Use the following pieces of context to answer the question at the end. Explain the rationale behind your answer.
If you don't have all the elements to answer the query, say it explicitly.

{context}

Question: {question}
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
from langchain.chat_models import ChatOpenAI
ask_me_about_fed_stuff = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name='gpt-3.5-turbo'),
                                                     chain_type="stuff",
                                                     retriever=vector_s.as_retriever(search_kwargs={"k": 7}),
                                                     return_source_documents=True,
                                                     chain_type_kwargs={'prompt': PROMPT})

In [ ]:
result = ask_me_about_fed_stuff({'query':'Views on monetary policy during March 26-27, 1984.'})
print(result['result'])

In [ ]:
result = ask_me_about_fed_stuff({'query':'Summarize the discussion around the selection of a reserve bank to operate the system account on March 26-27, 1984.'})
result['result']

In [ ]:
result = ask_me_about_fed_stuff({'query':'What was the economic outlook during the FOMC meeting of March 1968? Please include some of the political developments discussed in the meeting and why they were important to the economic outlook.'})
result['result']

In [ ]:
import re
pattern = re.compile(r'\d{8}')
dates =[pattern.findall(doc.metadata['source'])[0] for doc in result['source_documents']]
print(dates[1])

In [ ]:
def get_date(date: str):
    import pandas as pd
    date = pd.to_datetime(date)
    date = date.strftime("%d%m%Y")
    return date

print(get_date('15 Mar 1968'))




In [ ]:
query = 'hat was the economic outlook during the FOMC meeting of March 1968? Please include some of the political developments discussed in the meeting and why they were important to the economic outlook.?'
docs_ans = vector_s.similarity_search(query, k=7)

In [ ]:
docs_ans_meta = [doc.metadata['source'] for doc in docs_ans]
doc_dates = pattern.findall(doc.metadata['source'])[0]


### Getting the relevant dates

In [ ]:
from langchain.tools import tool

@tool
def parse_date(date: str):
    """
    Useful for when you need to search for the relevant date and parse it to
    """


In [ ]:
# Add a tool to filter the document store for those documents that contain the year information.
# Split into multiple vector stores or add filtering
# Add tools to search for different vector stores, one per year, and then combine them.

In [ ]:
dir(vector_s.docstore)

In [ ]:
vector_s.docstore.search('during')

### Getting the relevant dates

In [75]:
from langchain.tools import tool

@tool
def parse_date(date: str):
    """
    Useful for when you need to search for the relevant date and parse it to
    """


In [ ]:
# Add a tool to filter the document store for those documents that contain the year information.
# Split into multiple vector stores or add filtering
# Add tools to search for different vector stores, one per year, and then combine them.

In [101]:
dir(vector_s.docstore)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_dict',
 'add',
 'search']

In [103]:
vector_s.docstore.search('during')

'ID during not found.'